# PLN - Avaliação de Sentimentos V1

- Utilizando as bibliotecas NLTK e SKLearn demonstrar o uso de processamento de linguagem natural para avaliação do `Feedback` dos usuários

## 📦 1. Importar bibliotecas

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

- **NLTK**: para processamento de linguagem natural (tokenização, remoção de stopwords).
- **re**: para manipular texto (ex: remover pontuação).
- **TfidfVectorizer**: transforma textos em números (vetores) para o modelo.
- **MultinomialNB**: algoritmo de classificação baseado na probabilidade.
- **accuracy_score / classification_report**: avaliam o desempenho do modelo.

## 🔐 2. Baixar recursos necessários da NLTK

In [2]:
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to C:\Users\Luis
[nltk_data]     Rodrigo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Luis
[nltk_data]     Rodrigo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Luis
[nltk_data]     Rodrigo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

- `punkt`: usado para tokenizar palavras.
- `stopwords`: lista de palavras comuns (como "o", "a", "e") que não ajudam na análise.

## 📚 3. Dataset artificial

In [3]:
dados = [
    ("Eu amo programar em Python!", "positivo"),
    ("Python é incrível e muito útil.", "positivo"),
    ("Adorei este tutorial!", "positivo"),
    ("Não gosto desse sistema.", "negativo"),
    ("Essa experiência foi horrível.", "negativo"),
    ("Nunca mais usarei esse produto.", "negativo"),
    ("O suporte foi rápido e eficiente.", "positivo"),
    ("Tudo funciona bem aqui.", "positivo"),
    ("A interface é confusa.", "negativo"),
    ("Estou decepcionado.", "negativo"),
    ("Eu amo programar em Python!", "positivo"),
    ("Python é incrível e muito útil.", "positivo"),
    ("Adorei este tutorial!", "positivo"),
    ("Não gosto desse sistema.", "negativo"),
    ("Essa experiência foi horrível.", "negativo"),
    ("Nunca mais usarei esse produto.", "negativo"),
    ("O suporte foi rápido e eficiente.", "positivo"),
    ("Tudo funciona bem aqui.", "positivo"),
    ("A interface é confusa.", "negativo"),
    ("Estou decepcionado.", "negativo"),
    ("Adorei o filme!", "positivo"),
    ("Muito bom!","positivo" ),
    ("Excelente serviço!", "positivo"),
    ("Recomendo muito!", "positivo"),
    ("Ótima qualidade!", "positivo"),
    ("Maravilhoso!", "positivo"),
    ("Fantástico!", "positivo"),
    ("Incrível experiência!", "positivo"),
    ("Perfeito!", "positivo"), 
    ("Surpreendente!", "positivo"),
    ("Vale cada centavo!", "positivo"),
    ("Impecável!", "positivo"),
    ("Nota 10!", "positivo"),
    ("Superou expectativas!", "positivo"),
    ("Encantado!","positivo"),
    ("Sensacional!", "positivo"),
    ("Estou impressionado!", "positivo"),
    ("Melhor que esperava!", "positivo"),
    ("Divertido demais!", "positivo"),
    ("Amei!","positivo"),
    ("Top de linha!", "positivo"),
    ("Simplesmente perfeito!", "positivo"),
    ("Recomendo sem dúvidas!", "positivo"),
    ("Produto excelente!", "positivo"),
    ("Atendimento maravilhoso!","positivo"),
    ("Muito chato.", "negativo"), 
    ("Péssimo filme.", "negativo"), 
    ("Horrível experiência.", "negativo"), 
    ("Não recomendo.", "negativo"), 
    ("Terrível.", "negativo"), 
    ("Desapontado.", "negativo"), 
    ("Ruim demais.", "negativo"), 
    ("Lamentável.", "negativo"), 
    ("Fraco.", "negativo"), 
    ("Decepcionante.", "negativo"), 
    ("Perda de tempo.", "negativo"), 
    ("Não vale o preço.", "negativo"), 
    ("Horroroso.", "negativo"), 
    ("Péssimo atendimento.", "negativo"), 
    ("Arrependido.", "negativo"), 
    ("Muito ruim.", "negativo"), 
    ("Nunca mais compro.", "negativo"), 
    ("Qualidade inferior.", "negativo"), 
    ("Não funciona.", "negativo"), 
    ("Enganação.", "negativo"), 
    ("Pior experiência.", "negativo"), 
    ("Tive problemas.", "negativo"), 
    ("Não atendeu expectativas.", "negativo"), 
    ("Fiquei insatisfeito.", "negativo"), 
    ("Produto defeituoso.", "negativo")
]

- Cada entrada tem:
  - Um **texto** (opinião).
  - Um **rótulo** (`positivo` ou `negativo`).

## 🗂️ 4. Separar textos e rótulos

- `textos`: todas as frases.
- `rotulos`: os sentimentos associados.

In [7]:
textos = [t[0] for t in dados]
rotulos = [t[1] for t in dados]

print (f"\n\nTextos  : \n\t{textos[0:5]}")
print (f"\n\nRotulos : \n\t{rotulos[0:5]}")





Textos  : 
	['Eu amo programar em Python!', 'Python é incrível e muito útil.', 'Adorei este tutorial!', 'Não gosto desse sistema.', 'Essa experiência foi horrível.']


Rotulos : 
	['positivo', 'positivo', 'positivo', 'negativo', 'negativo']


## ✨ 5. Pré-processamento do texto

- **Limpa** o texto: remove pontuação, transforma em minúsculas.
- **Tokeniza**: divide o texto em palavras.
- **Remove stopwords**: elimina palavras irrelevantes como "o", "a", "e".

In [8]:
def preprocessar(texto):
    # Remover pontuação
    texto = re.sub(r'[^\w\s]', '', texto)
    
    # Tokenizar e converter para minúsculas
    tokens = word_tokenize(texto.lower())
    
    # Remover stopwords
    stop_words = set(stopwords.words('portuguese'))
    tokens_filtrados = [t for t in tokens if t not in stop_words]
    return ' '.join(tokens_filtrados)

## 🧹 6. Aplicar pré-processamento a todos os textos

- Retorna uma lista de textos já limpos e prontos para análise.

In [10]:
textos_limpos = [preprocessar(t) for t in textos]

print (f"\n\nTextos Limpos: \n\t{textos_limpos[0:5]}")



Textos Limpos: 
	['amo programar python', 'python incrível útil', 'adorei tutorial', 'gosto desse sistema', 'experiência horrível']


## 🤖 7. Vetorizar os textos com TF-IDF

- Transforma cada texto em um vetor numérico.
- Usa **TF-IDF**, que mede a importância das palavras no contexto

In [12]:
vectorizer = TfidfVectorizer()

## Obter as stopwords em português
#stopwords_pt = stopwords.words('portuguese')
#vectorizer = TfidfVectorizer(
#    max_features=5000,         # Limita o número máximo de palavras usadas
#    min_df=2,                  # Ignora palavras que aparecem em menos de 2 documentos
#    max_df=0.95,               # Ignora palavras muito comuns (>95% dos textos)
#    ngram_range=(1, 2),        # Considera bigramas além de unigramas
#    stop_words=stopwords_pt,   # Remove stopwords automaticamente
#    lowercase=True             # Converte tudo para minúsculas
#)

X = vectorizer.fit_transform(textos_limpos)


## 🧪 8. Dividir dados em treino e teste

- 80% dos dados são usados para **treinar** o modelo.
- 20% são usados para **testar** seu desempenho.

In [13]:
X_treino, X_teste, y_treino, y_teste = train_test_split(X, rotulos, test_size=0.2, random_state=42)

## 🧑‍💻 9. Treinar o modelo (Naive Bayes)'

- Cria um modelo Naive Bayer (MultinomialNB)
- Usa o algoritmo **Multinomial Naive Bayes**, bom para classificação de textos.
- Executa o modelo com os dados de treino (X_treino e y_treino)

In [16]:
modelo = MultinomialNB()
modelo.fit(X_treino, y_treino)

,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


## 🔮 10. Fazer previsões

- Submete os dados de de **teste** (`X_teste`)
- O modelo prevê os sentimentos dos textos do conjunto de **teste**.
- Obtem as **previsões** (`y_pred`)

In [17]:
y_pred = modelo.predict(X_teste)

## 📊 11. Avaliar o modelo

- **Acurácia**: percentual de acertos do modelo.
- **Classification report**: mostra métricas detalhadas (precisão, recall, f1-score) por classe.

In [19]:
print("Acurácia:", accuracy_score(y_teste, y_pred))
print("\nRelatório de classificação:")
print(classification_report(y_teste, y_pred))

Acurácia: 0.7142857142857143

Relatório de classificação:
              precision    recall  f1-score   support

    negativo       1.00      0.50      0.67         8
    positivo       0.60      1.00      0.75         6

    accuracy                           0.71        14
   macro avg       0.80      0.75      0.71        14
weighted avg       0.83      0.71      0.70        14



## ✅ Resumo Geral

| Etapa | O que faz |
|-------|-----------|
| 1-2   | Importar e preparar ferramentas |
| 3-4   | Criar e separar dataset |
| 5-6   | Limpar e preparar os textos |
| 7     | Converter textos em números |
| 8     | Dividir dados em treino/teste |
| 9     | Treinar modelo de classificação |
| 10-11 | Testar e avaliar o modelo |

Esse é um exemplo básico, mas completo, de como construir um **classificador de sentimentos** usando aprendizado de máquina. É muito útil para análises de opinião em redes sociais, avaliações de produtos, etc.

## 📁 Código Completo